In [1]:
import ast

In [2]:
from rich import print

In [8]:
tree = ast.parse("1+2")
print(ast.dump(tree, indent=4))

tree = ast.parse("1+2",mode="eval")
print(ast.dump(tree, indent=4))

Module(
    body=[
        Expr(
            value=BinOp(
                left=Constant(value=1),
                op=Add(),
                right=Constant(value=2)))],
    type_ignores=[])

Expression(
    body=BinOp(
        left=Constant(value=1),
        op=Add(),
        right=Constant(value=2)))

In [9]:
func_call_list = '[cd(folder="documents"), touch(file_name="data.csv")]'
print(ast.dump(
    ast.parse(func_call_list,mode="eval"),
    indent=4
    )
)

Expression(
    body=List(
        elts=[
            Call(
                func=Name(id='cd', ctx=Load()),
                args=[],
                keywords=[
                    keyword(
                        arg='folder',
                        value=Constant(value='documents'))]),
            Call(
                func=Name(id='touch', ctx=Load()),
                args=[],
                keywords=[
                    keyword(
                        arg='file_name',
                        value=Constant(value='data.csv'))])],
        ctx=Load()))

In [ ]:
def parse_agent_commands(code_str):
    """
    将工具调用字符串转换为结构化列表。
    输入: '[cd(folder="documents"), touch(file_name="data.csv")]'
    输出: [{"name": "cd", "arguments": {"folder": "documents"}}, ...]
    """
    # 1. 使用 eval 模式解析，获取 Expression 根节点
    tree = ast.parse(code_str, mode='eval')

    # 2. 检查根节点是否为 List (因为源码最外层是 [])
    if not isinstance(tree.body, ast.List):
        raise ValueError("输入格式错误：预期为一个指令列表 [cmd1(), cmd2()]")

    commands = []

    # 3. 遍历列表中的每一个元素 (elts)
    for elt in tree.body.elts:
        # 每一个元素必须是一个 Call (函数调用)
        if isinstance(elt, ast.Call):
            # 提取函数名 (对应 Call.func)
            # 这里假设 func 是简单的 Name 节点
            func_name = elt.func.id if isinstance(elt.func, ast.Name) else str(elt.func)

            # 提取关键字参数 (对应 Call.keywords)
            # 我们将其转换为 Python 字典
            args_dict = {}
            for kw in elt.keywords:
                # kw.arg 是参数名字符串, kw.value.value 是 Constant 的值
                args_dict[kw.arg] = kw.value.value

            # 构造结果
            commands.append({
                "name": func_name,
                "arguments": args_dict
            })

    return commands

# --- 测试一下 ---
raw_code = '[cd(folder="documents"), touch(file_name="data.csv")]'
result = parse_agent_commands(raw_code)

import json

print(json.dumps(result, indent=4))

In [2]:
import ast


def get_callable_expression(node):
    """
    解析 Call 节点的 func 属性，返回其可读的字符串表示。
    覆盖所有情况：Name, Attribute, Call, Subscript, Lambda, BinOp...
    """

    # 1. 基础情况：Name (直接调用)
    # 例子: print() -> "print"
    if isinstance(node, ast.Name):
        return node.id

    # 2. 递归情况：Attribute (点号调用)
    # 例子: os.path.join() -> "os.path.join"
    # 我们递归处理 value 部分，然后加上 attr
    elif isinstance(node, ast.Attribute):
        return f"{get_callable_expression(node.value)}.{node.attr}"

    # 3. 递归情况：Call (你的核心需求：链式调用/工厂模式)
    # 例子: get_model("bert")() -> "get_model('bert')(...)"
    # 这里的 node 本身就是一个 Call 节点，说明它是作为外层调用的 func 存在的
    elif isinstance(node, ast.Call):
        # 策略：还原内层函数调用的源码
        return ast.unparse(node)

    # 4. 兜底策略：处理 Subscript, Lambda, BinOp 等所有复杂情况
    # 例子: tools["search"]() -> "tools['search']"
    # 例子: (lambda x:x)() -> "(lambda x: x)"
    else:
        # ast.unparse 是 Python 3.9+ 的神器，它能把任何 AST 节点完美还原回代码字符串
        return ast.unparse(node)

# --- 压力测试 ---
test_cases = [
    ("print('hello')", "基础Name"),
    ("os.path.join('a', 'b')", "属性Attribute"),
    ("get_factory(mode='prod')('data')", "嵌套Call"),  # <-- 你关心的 Case
    ("tools['search'](query='hi')", "下标Subscript"),
    ("(lambda x: x+1)(10)", "匿名Lambda"),
    ("my_list[0].execute()", "组合拳"),
    ("(func_a if x else func_b)()", "条件表达式")
]

print(f"{'源码片段':<35} | {'解析出的调用者 (func)':<30} | 类型")
print("-" * 80)

for code, desc in test_cases:
    # 1. 拿到 AST
    tree = ast.parse(code, mode='eval')
    # 2. 提取外层 Call 的 func 节点
    # 注意：tree.body 是最外层的 Call，我们需要分析的是这个 Call 的 func 属性
    func_node = tree.body.func

    # 3. 解析
    result = get_callable_expression(func_node)
    print(f"{code:<35} | {result:<30} | {desc}")

源码片段                                | 解析出的调用者 (func)                 | 类型
--------------------------------------------------------------------------------
print('hello')                      | print                          | 基础Name
os.path.join('a', 'b')              | os.path.join                   | 属性Attribute
get_factory(mode='prod')('data')    | get_factory(mode='prod')       | 嵌套Call
tools['search'](query='hi')         | tools['search']                | 下标Subscript
(lambda x: x+1)(10)                 | lambda x: x + 1                | 匿名Lambda
my_list[0].execute()                | my_list[0].execute             | 组合拳
(func_a if x else func_b)()         | func_a if x else func_b        | 条件表达式
